In [ ]:
import pandas as pd
import numpy as np

# Load sample file
df = pd.read_csv('../data/glassdoor_sample_former_employees_100k.csv')
print('Shape before: ', df.shape)
df.head()

In [ ]:
df.dtypes

#### Data Profiling

- Temporary setting for column width 

In [ ]:
# ENABLE max column values display - for long text values
pd.set_option('display.max_colwidth', None)

- Empty columns

In [ ]:
# Find entirely empty columns
df.isna().sum().sort_values(ascending=False)

- Duplicates

In [ ]:
# Find entire row duplicates
duplicates = df.duplicated().sum()
print('Duplicate rows: ', duplicates)

- Column names

In [ ]:
df.columns.to_list()

In [ ]:
# Columns renamed prior to further profiling

print('Original columns: ', df.columns.to_list())

df.rename(columns={
    'rating': 'overall_rating',
    'Career Opportunities': 'career_opportunities_rating',
    'Compensation and Benefits': 'comp_benefits_rating',
    'Culture & Values': 'culture_values_rating',
    'Diversity & Inclusion': 'diversity_inclusion_rating',
    'job': 'job_title',
    'status': 'tenure_at_employer',
    'date': 'review_date',
    'title': 'review_title',
    'Senior Management': 'senior_management_rating',
    'Work/Life Balance': 'work_life_rating',
    'Recommend': 'recommend',
    'CEO Approval': 'ceo_approval',
    'Business Outlook': 'business_outlook',
    'firm_link' : 'company'
}, inplace=True)

print('New columns: ', df.columns.to_list())

- Columns that should be numeric dtype

In [ ]:
all_ratings_cols = ['overall_rating', 'diversity_inclusion_rating', 'culture_values_rating','work_life_rating', 'senior_management_rating', 'comp_benefits_rating', 'career_opportunities_rating']
df[all_ratings_cols].dtypes

- Columns that should be datetime dtype

In [ ]:
df['review_date'].dtypes

- Columns that should be string dtype

In [ ]:
# Inspect columns that should be string dtype
all_text_cols = ['review_title', 'company', 'job_title', 'pros', 'cons', 'ceo_approval', 'tenure_at_employer','recommend','business_outlook']
df[all_text_cols].dtypes

- 'pros' & 'cons' columns

In [ ]:
## Inspect Pros and Cons coment columns
comment_cols = ['pros', 'cons']
df[comment_cols]

- ratings columns

In [ ]:
# Inspect ratings columns
float_ratings = ['overall_rating', 'diversity_inclusion_rating', 'culture_values_rating']
print(df[float_ratings])

obj_ratings = ['work_life_rating', 'senior_management_rating', 'comp_benefits_rating', 'career_opportunities_rating']
print(df[obj_ratings])

for col in (float_ratings):
    print('FLOAT RATINGS: ', f"{col}: {df[col].unique()}") 


for col in (obj_ratings):
    print('OBJECT RATINGS: ', f"{col}: {df[col].unique()}")

- 'recommend', 'ceo_approval', 'business_outlook' columns

In [ ]:
## Inspect recommend|ceo_approval|business_outlook columns
opinion_map_cols = ['recommend', 'ceo_approval', 'business_outlook']
df[opinion_map_cols]

- 'tenure_at_employer' column

In [ ]:
## Inspect tenure column
print('Dtype: ', df['tenure_at_employer'].dtypes)

df['tenure_at_employer'].value_counts()

- company column

In [ ]:
## Inspect company column
df['company']

#### Data Cleaning

- Drop empty columns

In [ ]:
# Drop empty "index" column
df = df.drop(columns='index')

# Drop 'advice' column
df = df.drop(columns='advice')
df.columns
print(df.shape)

- Drop duplicates

In [ ]:
# Drop duplicates
df = df.drop_duplicates()
df.shape

- Convert all text columns to string dtype

In [ ]:
# objects to string
all_text_cols = ['review_title', 'company', 'job_title', 'pros', 'cons', 'ceo_approval', 'tenure_at_employer','recommend','business_outlook']

df[all_text_cols] = df[all_text_cols].astype('string')

# Verify dtype update to string
df[all_text_cols].dtypes

- Ratings columns

In [ ]:
# Get rid of non-numeric values in all ratings columns
obj_ratings = ['work_life_rating', 'senior_management_rating', 'comp_benefits_rating', 'career_opportunities_rating']
weird_values = ['10u0eun', 'e0wqkp','s4o194', '156pzk5']

# Replace weird values with NaN
df[obj_ratings] = df[obj_ratings].replace(weird_values, np.nan)


# Check columns unique values
for col in obj_ratings:
    print('New values: ', f"{col}: {df[col].unique()}")


In [ ]:
obj_ratings = ['work_life_rating', 'senior_management_rating', 'comp_benefits_rating', 'career_opportunities_rating']

# convert objects to floats
df[obj_ratings] = df[obj_ratings].astype('float')

for col in obj_ratings:
    print('Unique values: '' ', f"{col}: {df[col].unique()}")

df.info()

## Keep ratings NaNs!!  These inidicate no response, not actually 0 
# 'overall_rating' has no NaNs because mandatory

- DATE column

In [ ]:
# object to date
df['review_date'] = pd.to_datetime(df['review_date'], errors='coerce')

# Confirm new datetime dtype
df['review_date'].info()

- TEXT columns

- review_title

In [ ]:
# Capture # of nulls before changes
print('Null before: ',df['review_title'].isna().sum())

# Fill nulls with "Not provided"
df['review_title'] = df['review_title'].fillna('Not provided')

# Verify no more nulls
print('Nulls after: ', df['review_title'].isna().sum())

- business_outlook, recommend, ceo_approval columns

In [ ]:
# map business_outlook, recommend, ceo_approval values to labels
new_opp_vals = { 'v' : 'Positive', 'r': 'Mild', 'x': 'Negative', 'o' : 'No opinion'}
df[opinion_map_cols] = df[opinion_map_cols].replace(new_opp_vals)

# Verify changed values
df[opinion_map_cols]

- tenure_at_employer column

In [ ]:
# Re-bucket tenure lengths
df['tenure_at_employer'] = df['tenure_at_employer'].replace({
    'Former Employee' : 'Not provided',
    'Former Employee, less than 1 year' : 'Under 1 year',
    'Former Employee, more than 1 year' : '1-5 years',
    'Former Employee, more than 3 years' : '1-5 years',
    'Former Employee, more than 5 years' : '6-10 years',
    'Former Employee, more than 8 years' : '6-10 years',
    'Former Employee, more than 10 years' : 'Over 10 years'

})

# Set bucket sorting order
tenure_order = [
    'Under 1 year',
    '1-5 years',
    '6-10 years',
    'Over 10 years',
    'Not provided'
]

# Change to categorical to enforce sorting order
df['tenure_at_employer'] = pd.Categorical(
    df['tenure_at_employer'],
    categories=tenure_order,
    ordered=True
)

- company column

In [ ]:
# Replace NaNs with "Not Provided"
df['company'] = df['company'].fillna('Not provided')

# Extract company name
df['company'] = df['company'].str.extract(r'Reviews/([^/]+?)-Reviews', expand=False)

# Remove dashes between words
df['company'] = df['company'].str.replace('-', ' ', regex=False)

# Inspect company
df['company']

#### Revert any temporary settings

In [ ]:
# END max column display
pd.reset_option('display.max_colwidth')

#### Cleaned dataset

In [ ]:
# Export to csv
df.to_csv('cleaned_glassdoor_sample_data.csv', index=False)

# Preview sample of new csv
cleaned_sample = pd.read_csv('cleaned_glassdoor_sample_data.csv')
print(cleaned_sample.shape)
cleaned_sample.head()